<a href="https://colab.research.google.com/github/Samin-Sadaf7/MasteringObjectDetection/blob/main/MasteringObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Object Detection API
The course I am following: https://www.youtube.com/playlist?list=PLLLR8OQ3THK4Ff5Qs8fehzYLN73-VE3BA

Part 1: Data Processing

*   Install the necessary packages
*   Import the packages as well



In [2]:
# for interacting with operating system
import os
import pathlib
# Get current working dir
current_dir= os.getcwd()
print("Root dir:", current_dir)

#Clone tensorflow github model repository if it does not exist
if 'models' in pathlib.Path.cwd().parts:
  while 'models' in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Root dir: /content
Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3076/3076), done.
remote: Total 4084 (delta 1185), reused 1992 (delta 948), pack-reused 0
Receiving objects: 100% (4084/4084), 44.62 MiB | 14.21 MiB/s, done.
Resolving deltas: 100% (1185/1185), done.


In [3]:
#install Object Detection API
%cd $current_dir/models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install .

/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.1 MB/s eta 0:00:00
     ━━━━

In [1]:
import ast # evaluat input and covert to a data structure
import warnings # used to ignore some warnings
import numpy as np
import pandas as pd

import tensorflow as tf

#Data Visualization
import seaborn as sb
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#ignore all warnings
warnings.simplefilter('ignore')
#Draw in ipython env
%matplotlib inline